# data.preprocess

> Fill in a module description here

In [ ]:
#| default_exp data.preprocess

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import gc
from tqdm.notebook import tqdm

In [ ]:
#| export
def weekday_average(data:dict # pandas dictionnary containing the data
                   )->list[float]:
    "compute the weekday average"
    weekday_avgs = data.groupby(data.index.weekday).mean() * 24
    return weekday_avgs.values.flatten()

In [ ]:
#| export
def day_segment_average(data:dict # pandas dictionnary containing the data
                       )->list[float]:
    "compute the daily average"
    segments = {
        'early_morning': (7, 9),
        'morning': (9, 13),
        'early_afternoon': (13, 17),
        'late_afternoon': (17, 21),
        'night': [(21, 24), (0, 7)]
    }
    averages = []
    for segment, hours in segments.items():
        if segment == 'night':
            energy = data.between_time('21:00', '23:59').mean() * 10 + data.between_time('00:00', '06:59').mean() * 10
        elif  segment == 'early_morning':
            start, end = hours
            energy = data.between_time(f'{start}:00', f'{end-1}:59').mean() * 2
        else :
            start, end = hours
            energy = data.between_time(f'{start}:00', f'{end-1}:59').mean() * 4
            
        averages.append(energy)
    return np.array(averages).flatten()

In [ ]:
#| export
def total_energy_used(data:dict # pandas dictionnary containing the data
                     )->float:
    return data.sum()

In [ ]:
#| export
def average_energy_used(data:dict # pandas dictionnary containing the data
                       )->list[float]:
    "compute the average energy used"
    hourly_avg = data.mean()
    daily_avg = data.resample('D').sum().mean()
    weekly_avg = data.resample('W').sum().mean()
    monthly_avg = data.resample('M').sum().mean()
    return np.array([hourly_avg, daily_avg, weekly_avg, monthly_avg]).flatten()

In [ ]:
#| export
def weekend_businessday_avg(data:dict # pandas dictionnary containing the data
                           )->list[float]:
    "compute the weekend average"
    weekends_avg = data[data.index.weekday >= 5].resample('D').sum().mean() 
    business_days_avg = data[data.index.weekday < 5].resample('D').sum().mean()
    return np.array([weekends_avg, business_days_avg]).flatten()

In [ ]:
#| export
def resample_building_data(group:dict # pandas dictionnary containing the building data
                          )->dict:
    "resample the building data"
    group = group.reset_index(level='ID')
    # Specify columns explicitly for summing
    resampled_group = group.resample('h').agg({'consumption': 'sum'})  # Example if 'consumption' is your numeric column
    resampled_group['ID'] = group['ID'].iloc[0]  # Handle non-numeric separately if needed
    resampled_group = resampled_group.set_index('ID', append=True)
    return resampled_group

In [ ]:
#| export
def optimize_data_types(df):

    for col in df.select_dtypes(include=['int']).columns:
        df[col] = df[col].astype('int32')
    
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = df[col].astype('float32')

    for col in df.select_dtypes(include=['object']).columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')
    
    return df

In [ ]:
#| export
def create_lagged_features(df, window_size=40, start_date='2009-07-14 00:00:00'):

    data = []
    columns = [f'lag_{i}' for i in range(1, window_size+1)] + ['year', 'month', 'day', 'hour', 'target']
    base_date = pd.Timestamp(start_date)

    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        values = row.values
        for t in range(window_size, len(values)):
            lagged_inputs = values[t-window_size:t]
            target_value = values[t]

            # Calculate the corresponding date and time
            timestamp = base_date + pd.Timedelta(hours=t)
            year, month, day, hour = timestamp.year, timestamp.month, timestamp.day, timestamp.hour

            data.append(list(lagged_inputs) + [year, month, day, hour, target_value])

    df_transformed = pd.DataFrame(data, columns=columns)
    return df_transformed

In [ ]:
#| export
def create_lagged_features(df, window_size=40, start_date='2009-07-14 00:00:00'):

    data = []
    columns = [f'lag_{i}' for i in range(1, window_size+1)] + ['year', 'month', 'day', 'hour', 'target']
    base_date = pd.Timestamp(start_date)

    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        values = row.values
        for t in range(window_size, len(values)):
            lagged_inputs = values[t-window_size:t]
            target_value = values[t]

            # Calculate the corresponding date and time
            timestamp = base_date + pd.Timedelta(hours=t)
            year, month, day, hour = timestamp.year, timestamp.month, timestamp.day, timestamp.hour

            data.append(list(lagged_inputs) + [year, month, day, hour, target_value])

    df_transformed = pd.DataFrame(data, columns=columns)
    return df_transformed

In [ ]:
#| export
def preprocess_dataframe(df, window_size=40, chunk_size=100):

    # Optimize data types
    df = optimize_data_types(df)
    
    # Remove the first and last columns
    df = df.iloc[:, 1:-1]

    # Process the data in chunks to create lagged features
    processed_data = []
    for start_row in tqdm(range(0, df.shape[0], chunk_size)):
        chunk = df.iloc[start_row:start_row + chunk_size]
        df_chunk_transformed = create_lagged_features(chunk, window_size=window_size)
        processed_data.append(df_chunk_transformed)

    df_transformed = pd.concat(processed_data, ignore_index=True)

    # Make sure year, month, day, and hour columns are integers
    for col in ['year', 'month', 'day', 'hour']:
        df_transformed[col] = df_transformed[col].astype(int)

    # Create datetime column
    df_transformed['date_time'] = pd.to_datetime(df_transformed[['year', 'month', 'day', 'hour']])

    # Apply OrdinalEncoder to the month, day, and hour columns
    ordinal_encoder = OrdinalEncoder()
    df_transformed[['month', 'day', 'hour']] = ordinal_encoder.fit_transform(df_transformed[['month', 'day', 'hour']])

    return df_transformed

In [ ]:
#| export
def scale_features(train, test, columns_to_scale):

    scaler = StandardScaler()

    # Fit scaler on training data and apply the same transformation to both train and test data
    train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
    test[columns_to_scale] = scaler.transform(test[columns_to_scale])

    return train, test

In [ ]:
#| export
def finalize_dataframe(df_transformed, train_start_date, train_end_date, test_start_date, test_end_date):

    # Create the datetime index and sort
    df_transformed.set_index('date_time', inplace=True)
    df_transformed = df_transformed.sort_index()

    # Split into training and testing sets
    train = df_transformed.loc[train_start_date:train_end_date]
    test = df_transformed.loc[test_start_date:test_end_date]

    # Drop unnecessary columns like year, month, day, hour, and target
    #train = train.drop(columns=['year', 'month', 'day', 'hour', 'target'])
    #test = test.drop(columns=['year', 'month', 'day', 'hour', 'target'])

    return train, test

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()